In [18]:
import pandas as pd
import numpy as np
import pickle

In [19]:
data = pd.read_csv("data/test.csv")
names = data["Name"]
ids = data["PassengerId"]
data = data.drop(["Name", "PassengerId"], axis = 1)
data["Sex"] = data["Sex"].map({"male": 1, "female": 0})
data["Embarked"] = data["Embarked"].map({"S": 0, "C": 1, "Q": 2})
data = data.fillna(-1)
# data = data.drop(["Cabin", "Ticket"], axis = 1)
# data = data.dropna()

In [20]:
data

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,1,34.5,0,0,330911,7.8292,-1,2
1,3,0,47.0,1,0,363272,7.0000,-1,0
2,2,1,62.0,0,0,240276,9.6875,-1,2
3,3,1,27.0,0,0,315154,8.6625,-1,0
4,3,0,22.0,1,1,3101298,12.2875,-1,0
...,...,...,...,...,...,...,...,...,...
413,3,1,-1.0,0,0,A.5. 3236,8.0500,-1,0
414,1,0,39.0,0,0,PC 17758,108.9000,C105,1
415,3,1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,-1,0
416,3,1,-1.0,0,0,359309,8.0500,-1,0


In [21]:
def normalize_ticket(word):
    if word == "LINE":
        return -1
    word = word.split()[-1]
    word = int(word)
    return word

for i, elem in enumerate(data["Ticket"]):
    data.loc[i, "Ticket"] = normalize_ticket(elem)

In [22]:
from string import ascii_uppercase as alphabet
alphabet = list(alphabet)

def normalize_cabin(word):
    if word == -1:
        return -1
    word = word.split()
    result = []
    for w in word:
        elem = str(alphabet.index(w[0])) + w[1:]
        result.append(int(elem))
#     return ' '.join(result)
    return np.mean(result)

In [23]:
for i, elem in enumerate(data["Cabin"]):
    data.loc[i, "Cabin"] = normalize_cabin(elem)
data["Cabin"] = data["Cabin"].astype("float64")

In [25]:
data

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,1,34.5,0,0,330911,7.8292,-1.0,2
1,3,0,47.0,1,0,363272,7.0000,-1.0,0
2,2,1,62.0,0,0,240276,9.6875,-1.0,2
3,3,1,27.0,0,0,315154,8.6625,-1.0,0
4,3,0,22.0,1,1,3101298,12.2875,-1.0,0
...,...,...,...,...,...,...,...,...,...
413,3,1,-1.0,0,0,3236,8.0500,-1.0,0
414,1,0,39.0,0,0,17758,108.9000,2105.0,1
415,3,1,38.5,0,0,3101262,7.2500,-1.0,0
416,3,1,-1.0,0,0,359309,8.0500,-1.0,0


In [26]:
data["Cabin"].value_counts()

-1.00       327
 231.00       3
 161.25       3
 2101.00      2
 145.00       2
           ... 
 126.00       1
 3.00         1
 343.00       1
 286.00       1
 169.00       1
Name: Cabin, Length: 75, dtype: int64

# Loading model, predicting and writing predictions

In [34]:
filename = "rf.sav"
bst = pickle.load(open(filename, "rb"))

In [35]:
preds = bst.predict(data)
df = pd.DataFrame({"PassengerId": ids, "Survived": preds})
df

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 321 out of 321 | elapsed:    0.0s finished


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [36]:
df.to_csv("Submit.csv", index = False)